### Additionally installed libraries

In [ ]:
### Install dependencies:
%%bash
git clone https://github.com/open2c/pairtools.git
cd pairtools
pip install -e .

pip install bioframe pandas cooler

In [ ]:
### Download input data:
! wget https://gitlab.rlp.net/3d-diploid-chromatin/simulation-code/-/raw/main/Simulation_Data/Data/gm12878_17.pairs

### Import dependencies

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pairtools.lib import headerops, fileio

In [49]:
import bioframe
import pandas as pd
import pairtools
import numpy as np
import cooler

In [41]:
# Get chromosome sizes:
chromsizes = bioframe.fetch_chromsizes(genome)[:'chrY']

In [67]:
def phase_chrom_sizes(chromsizes, phases=None):
    """
    Get the dataframe of chromosome sizes for hap- or di-ploid genomes.
    """
    output = pd.DataFrame()
    for phase in phases:
        output_frag = chromsizes.reset_index()
        output_frag.loc[:, 'name'] = output_frag['name'].apply(lambda x: f"{x}_{phase}")
        output = pd.concat([output, output_frag])

    output = output.set_index('name')
    return output

def get_beads_per_chromosomes(chromsizes, resolution):
    output = np.ceil( chromsizes/resolution ).astype(np.int_)
    return output

In [68]:
# Phase chromosome sizes, create diploid genome:
chromsizes_diploid = phase_chrom_sizes(chromsizes, phases=('0', '1'))

# Convert genome to the bin sizes:
resolution = 10_000
chromsizes_res     = get_beads_per_chromosomes(chromsizes_diploid, resolution)

In [111]:
# Create diploid viewframe on the genome:
genome_viewframe_diploid = bioframe.make_viewframe(chromsizes_diploid['length'])

In [118]:
# Create bin table:
bin_table = cooler.binnify(chromsizes_diploid['length'], resolution)

### Read contact pairs

In [75]:
# Read scHi-C interaction pairs:
pairs_file = "./gm12878_17.pairs"
genome = "hg19"

pairs_stream = fileio.auto_open(pairs_file, 'r')
header, pairs_stream = headerops.get_header(pairs_stream)
columns = headerops.get_colnames(header)

df = pd.read_table(pairs_stream, comment="#", header=None)
df.columns = columns

# Modify chromosome names:
df.loc[:, 'chr1'] = df.loc[:, 'chr1'].apply(lambda x: f'chr{x}')
df.loc[:, 'chr2'] = df.loc[:, 'chr2'].apply(lambda x: f'chr{x}')

/tmp/ipykernel_1189863/3026421978.py:10: DtypeWarning: Columns (1,3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_table(pairs_stream, comment="#", header=None)


### Run for single round of simualtions

In [128]:
def split_pairs(df, 
                phase_cols=('phase0', 'phase1'),
                accepted_phases=('0', '1')
               ):
    """Split the pairs into ambiguous and non-ambiguous pairs"""

    mask = np.ones(len(df), dtype=np.bool_)
    for phase_col in phase_cols:
        mask &= np.in1d( df.loc[:, phase_col], accepted_phases )
    return df[mask], df[~mask]

def annotate_annotated(df, bin_table):
    df = df.copy()
    for side in [1, 2]:
    
        df_tmp = df.loc[:, [f'chr{side}', f'phase{side-1}', f'pos{side}']].copy()
        df_tmp.loc[:, 'chrom'] = df_tmp.apply(lambda x: 
                                        f"{x[f'chr{side}']}_{x[f'phase{side-1}']}", axis=1)
        df_tmp.loc[:, 'start'] = df_tmp[f'pos{side}'].copy()
        df_tmp.loc[:, 'end']   = df_tmp[f'pos{side}'].copy()+1
        
        index = bioframe.overlap(df_tmp, bin_table, return_index=True)['index_']
        df.loc[:, f'bin_index_{side}'] = index

    return df

def get_edges(df, cols=['bin_index_1', 'bin_index_2']):
    df_unique = df.dropna(subset=cols).drop_duplicates(subset=cols)
    return df_unique[cols]

In [144]:
# Split pairs into ambiguous vs not: 
df_accepted, df_ambiguous = split_pairs(df)
print('Ambiguous vs resolved:', len(df_ambiguous), len(df_accepted))

Ambiguous vs resolved: 1826692 12546


In [136]:
# Annotate accepted pairs by bins:
df_accepted = annotate_annotated(df_accepted, bin_table)

# Get the edges: 
df_edges = get_edges(df_accepted)

In [146]:
df_accepted

,readID,chr1,pos1,chr2,pos2,strand1,strand2,phase0,phase1,phase_prob00,phase_prob01,phase_prob10,phase_prob11,pos1_bin,pos2_bin
110,.,chr1,1001028,chr1,2240428,+,-,0,0,1.0,0.0,0.0,0.0,100,224
403,.,chr1,1545254,chr1,1549910,-,+,0,0,1.0,0.0,0.0,0.0,154,154
845,.,chr1,2319499,chr1,2353914,-,+,0,0,1.0,0.0,0.0,0.0,231,235
985,.,chr1,2757599,chr1,2761676,-,+,0,0,1.0,0.0,0.0,0.0,275,276
987,.,chr1,2757810,chr1,2761518,+,-,0,0,1.0,0.0,0.0,0.0,275,276
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1835761,.,chrX,142432041,chrX,142512217,-,+,0,0,1.0,0.0,0.0,0.0,14243,14251
1836877,.,chrX,146513228,chrX,146582025,+,+,0,0,1.0,0.0,0.0,0.0,14651,14658
1836878,.,chrX,146513439,chrX,146581814,-,-,0,0,1.0,0.0,0.0,0.0,14651,14658
1837989,.,chrX,150415430,chrX,150422686,-,-,0,0,1.0,0.0,0.0,0.0,15041,15042


In [ ]:
#### Next steps:
# Go from input 3D genome to the set of isolated, resolved and ambiguous contacts